In [6]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
import torch
import misc.util as util
from torch import nn

import importlib
from torch.utils.data import DataLoader
from misc.loader import AEDataset
from misc.util import *
from tqdm.auto import tqdm
from eval.rec_eval import *
import neuralsort.pl as pl
from models.loss import neuPrecLoss
from misc.loader import RecDataset
import models
from models.loss import *
from scipy.sparse import *

In [7]:
import pickle
with open("data/parsed/ml-20m-new", 'rb') as f:
    (tr_users, val_users, te_users, train_data, val_tr, val_te, te_tr, te_te) =pickle.load(f)

In [8]:
n_users, n_items = train_data.shape
ae_dataset = AEDataset(train_data[tr_users])
empty = csr_matrix(train_data.shape)

In [9]:
vin = val_tr[val_users]
vo = val_te[val_users]

In [10]:
vo.shape

(10000, 20108)

In [11]:
def MultinomialLoss(y, y_hat):
    return -(torch.log_softmax(y_hat, -1) * y).sum(-1)

In [12]:
MultinomialLoss(row, scores).mean()

NameError: name 'row' is not defined

In [13]:
s = scores[:5, :5]
y = row[:5, :5]

NameError: name 'scores' is not defined

In [14]:
-(torch.log_softmax(scores[:5, :5], -1) * row[:5, :5]).sum(-1).mean()

NameError: name 'scores' is not defined

In [43]:
def MultinomialLoss(y, s):
    return -(torch.log(torch.softmax(s, -1)) * y).sum(-1).mean()

In [54]:
(torch.log(torch.softmax(s, -1)) * y.bool()).sum(-1).mean()

tensor(-0.9181, device='cuda:0', grad_fn=<MeanBackward0>)

In [117]:
model.layers
model.train()

MultiDAE(
  (layers): ModuleList(
    (0): Linear(in_features=20108, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=20108, bias=True)
  )
  (drop): Dropout(p=0.5, inplace=False)
)

In [119]:
import models.ae
importlib.reload(models.ae)
import neuralsort.neuralobjs
importlib.reload(models.ae)
batch_size=500
use_vae=False
loader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=True, num_workers=6)
if use_vae:
    model = models.ae.MultiVAE([256] + [n_items], dropout=0.5)
else:
    model = models.ae.MultiDAE([256] + [n_items], dropout=0.5)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0)
sc = neuralsort.neuralobjs.SC()
lm = -1 

update_count = 0
total_anneal_steps = 200000
anneal_cap = 0.3
for epoch in (range(1, 5 + 1)):
    tot_loss = 0
    tot_loss = 0
    model = model.train()
    model.training = True
    # train for one epoch
    for uid, rowl in tqdm(loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        if use_vae:
            scores, mu, logvar = model.forward(row)
            if total_anneal_steps > 0:
                anneal_cap = min(anneal_cap, 1. * update_count / total_anneal_steps)
            else:
                anneal = anneal_cap
            loss  = models.ae.loss_function(scores, row, mu, logvar).mean()
            update_count += 1
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
        else:
            scores = model.forward(row)
            loss  = models.loss.MultinomialLoss(row, scores).mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)

        tot_loss += loss.detach().cpu().numpy()
        optimizer.step()
    
    print(batch_size * tot_loss / n_users)
    if (epoch % 5 == 0):
        model.eval()
        model.training=False
        wrapper = models.ae.implicitWrapper(model, vin, naive_sparse2tensor, vae=True)
        pr = (ranking_metrics_at_k(wrapper, vin, vo, K=5, num_threads=4)['precision'])
        x = ranking_metrics_at_k(wrapper, vin, vo, K=10, num_threads=4)
        print(pr, x['map'], x['ndcg'])


476.2286775244395



432.7242736608754



425.2861407720735



420.54658529992673



417.2619549239065
0.2657000000000004 0.43510400554295936 0.3036994365363418


In [5]:
empty = csr_matrix(train_data.shape)

NameError: name 'train_data' is not defined